In [7]:
import pandas as pd
import numpy as np
import time
import scipy.stats as stats
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.linear_model import LinearRegression
from scipy import stats
from sklearn.metrics import log_loss
from sklearn.calibration import CalibratedClassifierCV, calibration_curve
import matplotlib.pyplot as plt
from sklearn.metrics import (brier_score_loss, precision_score, recall_score,
                             f1_score)
from sklearn.metrics import roc_auc_score
import pickle

In [6]:
data = pd.read_sas('/home/guangya/Downloads/wnv_2245new.sas7bdat') 

In [8]:
description = ['unique hexid used by arcgis',
 'unique id',
'weekly mean temperature (C)',
'weekly mean precipitation (mm)',
'year',
'week of the year (Jan 1 = week 1)',
'temperature lag 1 week',
'temperature lag 2 weeks',
'temperature lag 3 weeks',
'temperature lag 4 weeks',
'precipitation lag 1 week',
'precipitation lag 2 weeks',
'precipitation lag 3 weeks',
'precipitation lag 4 weeks',
'mean mosquito infection rate',
'mosquito infection rate lag 1 week',
'mosquito infection rate lag 2 weeks',
'mosquito infection rate lag 3 weeks',
'mosquito infection rate lag 4 weeks'
,'total human population',
'% white'
,'% black',
'% asian',
'% hispanic',
'average household income',
'open water percent',
'developed open space percent',
'developed low intensity percent',
'developed medium intensity percent',
'developed high intensity percent',
'barren land percent',
'deciduous forest percent',
'evergreen forest percent',
'mixed forest percent',
'shrub percent',
'grassland percent',
'pasture percent',
'cultivated crops percent',
'woody wetlands percent',
'emergent herbaceous wetlands percent',
'mean temperature in January',
'N/A',
'housing % pre world-war 2',
'housing % post world-war 2 (1945-1969)',
'housing % 1970-1989',
'housing % post 1990',
              'Average household Income(K)']

In [9]:
model_1_max = pickle.load(open('/home/guangya/Downloads/RF_model_max', 'rb'))
features = data.columns.drop(['yrweeks', 'yrwksfid', 'yr_hexid', 'year', 'wnvbinary'])
data = pd.read_sas('/home/guangya/Downloads/wnv_2245new.sas7bdat') 
ip = model_1_max.best_estimator_.feature_importances_
s = np.array([features,description,ip]).T

In [ ]:
temp = pd.DataFrame(s,columns=['feature_name','description','feature_importance'])
temp.to_csv('temp.csv') # Plot for Modelling 1 first model

In [90]:
# wait for 2nd model
model_1_2 = pickle.load(open('/home/guangya/Downloads/RF_model_1_2_CV_final_02142020', 'rb'))

In [93]:
ip1_2 = model_1_2.best_estimator_.feature_importances_

In [103]:
np.array(['tempc', 'preci', 'templag1', 'templag2',
       'templag3', 'templag4', 'precilag1', 'precilag2', 'precilag3',
       'precilag4', 'mirmean', 'mirlag1', 'mirlag2', 'mirlag3', 'mirlag4',
       'totpop', 'dlipct', 'dmipct', 'Jantemp', 'hpctpreww', 'hpctpostww',
       'hpct7089'])[np.where(ip1_2 > ip1_2.mean() + 0.3 * ip1_2.std())]

array(['templag1', 'templag2', 'templag4', 'mirlag1', 'mirlag2',
       'mirlag3', 'mirlag4', 'totpop'], dtype='<U10')

In [104]:
model_1_2.best_params_

{'bootstrap': True,
 'max_depth': 80,
 'max_features': 'sqrt',
 'min_samples_leaf': 50,
 'n_estimators': 1300}

In [ ]:
# wait for 3rd model

In [23]:
#ks test for modelling 1
x = data.drop(columns = ['yrweeks', 'yrwksfid', 'yr_hexid', 'year'])
agg_wnv = x[x.wnvbinary == 1].drop(columns = 'wnvbinary')
agg_0 = x[x.wnvbinary == 0].drop(columns = 'wnvbinary')
agg_0 = agg_0[(agg_0.weeks >= 22) & (agg_0.weeks <= 31)]
agg_wnv = agg_wnv[(agg_wnv.weeks >= 22) & (agg_wnv.weeks <= 31)]
features_ks1 = []
p_ks1 = []
for x in agg_0.columns:
    print(x + ': ' + str(stats.ks_2samp(agg_wnv[x], agg_0[x])[1]))
    value = stats.ks_2samp(agg_wnv[x], agg_0[x])[1]
    features_ks1.append(x)
    p_ks1.append(value)

hexid: 6.390213842202285e-07
Input_FID: 6.390213842202285e-07
tempc: 3.529596088854107e-38
preci: 6.950997568998862e-06
yr: 4.034872356380986e-09
weeks: 3.3018410530629804e-42
templag1: 1.6075329481971952e-40
templag2: 6.524223156077696e-46
templag3: 2.3985008414230865e-40
templag4: 1.5088000766536422e-37
precilag1: 0.027885493943262542
precilag2: 0.5608123364427637
precilag3: 3.745945877464269e-11
precilag4: 3.132445075682942e-24
mirmean: 1.4549509581966023e-54
mirlag1: 1.9366416927172542e-48
mirlag2: 7.85066387729148e-42
mirlag3: 4.015931957483083e-41
mirlag4: 2.2927963593503358e-33
totpop: 1.136967814324584e-19
whitepct: 0.1798101789089166
blackpct: 0.11853166375855227
asianpct: 0.10793052934232421
hispanicpct: 0.02458563452262036
Income: 0.14265365400704844
owpct: 0.0017572775675292016
dospct: 5.3951598287421916e-05
dlipct: 4.3320124780038627e-07
dmipct: 0.017010620858381828
dhipct: 0.00045290546275062425
blpct: 0.9999995574048837
dfpct: 4.11509493679482e-05
efpct: 0.99686684201835

In [24]:
s_1ks = np.array([features_ks1,description,p_ks1]).T

In [26]:
temp = pd.DataFrame(s_1ks,columns=['feature_name','description','p_value'])

In [28]:
temp.to_csv('temp.csv')

In [29]:
# Modeeling2 model 1
model_2_max = pickle.load(open('/home/guangya/Downloads/RF_model_CV_final_1_22_2020', 'rb'))


In [30]:
ip_2_1 = model_2_max.best_estimator_.feature_importances_

In [31]:
len(ip_2_1)

44

In [42]:
features_2_1 = features.drop(features[[0,-1,-6]])

In [53]:
temp_2_1 = temp.drop([0,46,41])
s = np.array([temp_2_1.feature_name.values,temp_2_1.description.values,ip_2_1]).T

In [54]:
s = np.array([temp_2_1.feature_name.values,temp_2_1.description.values,ip_2_1]).T
temp_2_1 = pd.DataFrame(s,columns=['feature_name','description','feature_importance'])
temp_2_1.to_csv('temp.csv') # Plot for Modelling 1 first model

In [55]:
temp_2_1

,feature_name,description,feature_importance
0,Input_FID,unique id,0.0307242
1,tempc,weekly mean temperature (C),0.0352794
2,preci,weekly mean precipitation (mm),0.029424
3,yr,year,0.00500553
4,weeks,week of the year (Jan 1 = week 1),0.00282802
5,templag1,temperature lag 1 week,0.0351095
6,templag2,temperature lag 2 weeks,0.0329033
7,templag3,temperature lag 3 weeks,0.0383755
8,templag4,temperature lag 4 weeks,0.0508218
9,precilag1,precipitation lag 1 week,0.0294324


In [57]:
# Modeeling2 model 2
model_2_second  = pickle.load(open('/home/guangya/Downloads/RF_model_CV_final_1_30_2020', 'rb'))

In [59]:
ip_2_2 = model_2_second.best_estimator_.feature_importances_

In [83]:
columns = ['tempc', 'preci', 'templag1', 'templag2', 'templag3', 'templag4',
       'precilag1', 'precilag2', 'precilag3', 'precilag4', 'mirmean',
       'mirlag1', 'mirlag2', 'mirlag3', 'mirlag4', 'totpop', 'whitepct',
       'blackpct', 'asianpct', 'Income', 'dlipct', 'dmipct', 'dhipct',
       'Jantemp', 'hpctpreww', 'hpctpostww', 'hpct7089', 'hpctpost90']
temp_2_2 = temp_2_1[temp_2_1.feature_name.isin(columns)]

In [84]:
temp_2_2

,feature_name,description,feature_importance
1,tempc,weekly mean temperature (C),0.0352794
2,preci,weekly mean precipitation (mm),0.029424
5,templag1,temperature lag 1 week,0.0351095
6,templag2,temperature lag 2 weeks,0.0329033
7,templag3,temperature lag 3 weeks,0.0383755
8,templag4,temperature lag 4 weeks,0.0508218
9,precilag1,precipitation lag 1 week,0.0294324
10,precilag2,precipitation lag 2 weeks,0.0375501
11,precilag3,precipitation lag 3 weeks,0.0357219
12,precilag4,precipitation lag 4 weeks,0.0296077


In [85]:
s = np.array([temp_2_2.feature_name.values,temp_2_2.description.values,ip_2_2]).T
temp_2_2 = pd.DataFrame(s,columns=['feature_name','description','feature_importance'])
temp_2_2.to_csv('temp.csv') # Plot for Modelling 1 first model

In [71]:
temp.feature_name[3] ==

False

In [86]:
model_2_final = pickle.load(open('/home/guangya/Downloads/RF_model_2_3_CV_final_02142020', 'rb'))

In [87]:
model_2_final.best_estimator_.feature_importances_

array([0.13302082, 0.19627448, 0.11363494, 0.13643918, 0.12484555,
       0.13749261, 0.15829243])

In [102]:
model_2_final.cv_results_

{'mean_fit_time': array([65.05606771, 64.3675024 , 64.21188154, 64.73817286, 64.29903469,
        64.26393228, 64.41943974, 64.35052514, 63.66350651]),
 'std_fit_time': array([1.64767479, 2.09835815, 1.75762286, 2.0995366 , 1.90480436,
        2.0919353 , 1.7660347 , 1.7586749 , 1.72126435]),
 'mean_score_time': array([0.91622329, 0.93924103, 0.89443154, 0.89548173, 0.91631169,
        0.89886022, 0.91969213, 0.99597321, 0.83352852]),
 'std_score_time': array([0.03518222, 0.00425029, 0.05067295, 0.0474201 , 0.03825356,
        0.04986971, 0.08058017, 0.0783262 , 0.00118591]),
 'param_bootstrap': masked_array(data=[True, True, True, True, True, True, True, True, True],
              mask=[False, False, False, False, False, False, False, False,
                    False],
        fill_value='?',
             dtype=object),
 'param_max_depth': masked_array(data=[50, 50, 50, 60, 60, 60, 70, 70, 70],
              mask=[False, False, False, False, False, False, False, False,
               